In [ ]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from MovieAnalysis import MovieAnalysis
movie = MovieAnalysis()

In [ ]:
movie.movie_type(10)

In [ ]:
movie.movie_data

In [ ]:
movie.character_data

In [ ]:
actors_per_movie = movie.character_data.groupby('Freebase movie ID').agg(actor_count = ('Freebase actor ID', 'count'))


In [ ]:
actors_per_movie.groupby('actor_count').agg(movie_count = ('actor_count', 'count')).sort_values('movie_count', ascending=False)